In [ ]:
!pip install tensorflow_text

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [2]:
df = pd.read_csv("procData.csv")
df.head()

,title,files,description,category,fileAmount,more100Files,fileSize
0,[valoon] All 1982 1.Winter Anime Openings,[valoon] All 1982 1. Winter Anime Openings.mp4,\n Fresh start into 2022 on...,Anime-Anime Music Video,1,0,18.165656
1,[MTBB] GOTCHA! (BD 1080p),[MTBB] GOTCHA! [BD 1080p] [3071BC7C].mkv,\n A Blu-ray finally came o...,Anime-Anime Music Video,1,0,19.964831
2,animetronic - Sol Bianca Clip 'Club Cut' MV (D...,animetronic - Sol Bianca Clip 'Club Cut'.mkv,\n https://myanimelist.net/...,Anime-Anime Music Video,1,0,19.619635
3,[valoon] All 1992 1.Winter Anime Openings,[valoon] All 1992 1. Winter Anime Openings.mp4,"\n Finally, this is getting...",Anime-Anime Music Video,1,0,18.400905
4,[DB-MV]_NHK_Minna_no_Uta_-_Egao,[DB-MV]_NHK_Minna_no_Uta_-_Egao_[2868D747].avi,\n ```\nA music video direc...,Anime-Anime Music Video,1,0,18.021827


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70294 entries, 0 to 70293
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         70294 non-null  object 
 1   files         70294 non-null  object 
 2   description   70294 non-null  object 
 3   category      70294 non-null  object 
 4   fileAmount    70294 non-null  int64  
 5   more100Files  70294 non-null  int64  
 6   fileSize      70294 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 3.8+ MB


In [4]:
train, val, test = np.split(df.sample(frac=1), [int(0.8 * len(df)), int(0.9 * len(df))])

In [5]:
print(len(train), len(val), len(test), len(df))

56235 7029 7030 70294


In [6]:
from sklearn.utils import class_weight
# df['category'] = df['category'].astype('str')
class_weights = list(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df['category']), y=df['category']))

In [7]:
weights = {}
for index, w in enumerate(class_weights):
  weights[index] = w
weights 

{0: 59.926683716965044,
 1: 0.16583545265382965,
 2: 0.2113870035514845,
 3: 0.8522137626691237,
 4: 2.197099456148028,
 5: 3.701827373742693,
 6: 0.9706434686550677,
 7: 67.78592092574735,
 8: 0.6030248179190008,
 9: 3.9989759927181705,
 10: 22.9718954248366,
 11: 16.152113970588236,
 12: 0.8404351984696318,
 13: 44.461733080328905,
 14: 295.3529411764706,
 15: 153.1459694989107,
 16: 23.361249584579596}

In [8]:
np.unique(df['category'])

array(['Anime-Anime Music Video', 'Anime-English-translated',
       'Anime-Non-English-translated', 'Anime-Raw', 'Audio-Lossless',
       'Audio-Lossy', 'Literature-English-translated',
       'Literature-Non-English-translated', 'Literature-Raw',
       'Live Action-English-translated',
       'Live Action-Idol/Promotional Video',
       'Live Action-Non-English-translated', 'Live Action-Raw',
       'Pictures-Graphics', 'Pictures-Photos', 'Software-Applications',
       'Software-Games'], dtype=object)

In [9]:
def df_to_dataset(dataframe, shuffle=True, batch_size=100):
  df = dataframe.copy()
  labels = df.pop('category')
  df = {key: value[:,tf.newaxis] for key, value in df.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [10]:
train_data = df_to_dataset(train)
test_data = df_to_dataset(test)
validation_data = df_to_dataset(val)

<ipython-input-9-e90f28cd74b0>:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in df.items()}


In [11]:
train['category'].value_counts()

Anime-English-translated              19941
Anime-Non-English-translated          15603
Literature-Raw                         5437
Live Action-Raw                        3994
Anime-Raw                              3913
Literature-English-translated          3429
Audio-Lossless                         1495
Audio-Lossy                             901
Live Action-English-translated          837
Live Action-Non-English-translated      205
Live Action-Idol/Promotional Video      139
Software-Games                          131
Pictures-Graphics                        68
Anime-Anime Music Video                  60
Literature-Non-English-translated        47
Software-Applications                    22
Pictures-Photos                          13
Name: category, dtype: int64

In [12]:
catVals = np.unique(df['category'])
table = tf.lookup.StaticHashTable(
    initializer = tf.lookup.KeyValueTensorInitializer(
        keys = tf.constant(catVals),
        values = list(range(len(catVals)))
    ),
    default_value = -1,
    name = "target_encoding"
)

In [13]:
catVals

array(['Anime-Anime Music Video', 'Anime-English-translated',
       'Anime-Non-English-translated', 'Anime-Raw', 'Audio-Lossless',
       'Audio-Lossy', 'Literature-English-translated',
       'Literature-Non-English-translated', 'Literature-Raw',
       'Live Action-English-translated',
       'Live Action-Idol/Promotional Video',
       'Live Action-Non-English-translated', 'Live Action-Raw',
       'Pictures-Graphics', 'Pictures-Photos', 'Software-Applications',
       'Software-Games'], dtype=object)

In [14]:
@tf.function
def target(x):
  return table.lookup(x)

In [15]:
def fetch(features, labels):
  return features, tf.one_hot(target(labels), len(catVals))

In [16]:
train_data_f = train_data.map(fetch)
test_data_f = test_data.map(fetch)
validation_data_f = validation_data.map(fetch)

In [17]:
type(next(iter(train_data_f)))

tuple

In [18]:
embedding = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
hub_layer = hub.KerasLayer(embedding, output_shape = 512, input_shape = [], dtype=tf.string, trainable=True)

In [19]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = tf.keras.layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = tf.keras.layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = tf.keras.layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

In [20]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = tf.keras.layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [21]:
all_inputs = []
encoded_features = []

# Numerical features.
for header in ['fileAmount', 'fileSize']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_data_f)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [22]:
for header in ['more100Files']:
  num_cat_col = tf.keras.Input(shape=(1,), name=header)
  categ_layer = get_category_encoding_layer(header, train_data_f, "int64")
  encoded_categ_col = categ_layer(num_cat_col)
  all_inputs.append(num_cat_col)
  encoded_features.append(encoded_categ_col)

In [23]:
for header in ['title', 'description', 'files']:
  text_col = tf.keras.Input(shape=(), name=header, dtype='string')
  encoded_text_col = hub_layer(text_col)
  all_inputs.append(text_col)
  encoded_features.append(encoded_text_col)

In [24]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = all_features
for i in range(3):
  x = tf.keras.layers.Dense(16, activation='relu')(x)
  x = tf.keras.layers.Dropout(0.4)(x)
output = tf.keras.layers.Dense(len(catVals), activation='softmax')(x)

In [25]:
model = tf.keras.Model(all_inputs, output)

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 more100Files (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 fileAmount (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 fileSize (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 integer_lookup (IntegerLookup)  (None, 1)           0           ['more100Files[0][0]']           
                                                                                              

In [31]:
history = model.fit(train_data_f, epochs=10, validation_data=validation_data_f, class_weight=weights)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/keras/backend.py:5534: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


563/563 [==============================] - 523s 792ms/step - loss: 2.5487 - accuracy: 0.1076 - val_loss: 2.0890 - val_accuracy: 0.1131
Epoch 2/10
563/563 [==============================] - 233s 412ms/step - loss: 2.3916 - accuracy: 0.1800 - val_loss: 1.9835 - val_accuracy: 0.1117
Epoch 3/10
563/563 [==============================] - 215s 382ms/step - loss: 2.3623 - accuracy: 0.1856 - val_loss: 1.9656 - val_accuracy: 0.0956
Epoch 4/10
563/563 [==============================] - 207s 367ms/step - loss: 2.3284 - accuracy: 0.1911 - val_loss: 1.8740 - val_accuracy: 0.1151
Epoch 5/10
563/563 [==============================] - 203s 361ms/step - loss: 2.3184 - accuracy: 0.1873 - val_loss: 1.8643 - val_accuracy: 0.1009
Epoch 6/10
563/563 [==============================] - 201s 357ms/step - loss: 2.3163 - accuracy: 0.2172 - val_loss: 1.8483 - val_accuracy: 0.1154
Epoch 7/10
563/563 [==============================] - 201s 356ms/step - loss: 2.3106 - accuracy: 0.1873 - val_loss: 1.8542 - val_accura

In [32]:
model.evaluate(test_data_f)

71/71 [==============================] - 9s 120ms/step - loss: 1.8304 - accuracy: 0.1159


[1.8303556442260742, 0.11593171954154968]

In [33]:
model.evaluate(validation_data_f)

71/71 [==============================] - 9s 128ms/step - loss: 1.8264 - accuracy: 0.1152


[1.8263601064682007, 0.11523687839508057]

In [40]:
model.save("trainedModel_balanced")

In [35]:
test_x = test_data_f.unbatch().map(lambda x, y: x)
test_y = np.concatenate([y for x, y in test_data_f], axis=0)
test_predicted = model.predict(test_x)

7030/7030 [==============================] - 195s 27ms/step


In [36]:
type(next(iter(test_data_f)))

tuple

In [37]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [38]:
print(classification_report(test_y.argmax(1), test_predicted.argmax(1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00      2528
           2       0.00      0.00      0.00      1969
           3       0.08      0.03      0.04       467
           4       0.09      0.10      0.10       191
           5       0.03      0.06      0.04       109
           6       0.10      0.21      0.14       411
           7       0.00      0.00      0.00         8
           8       0.19      0.05      0.09       696
           9       0.00      0.00      0.00       103
          10       0.00      0.00      0.00        21
          11       0.00      0.86      0.01        21
          12       0.00      0.00      0.00       463
          13       0.00      0.00      0.00        13
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         3
          16       0.00      0.00      0.00        24

    accuracy              

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
print(tf.math.confusion_matrix(np.array(test_y).argmax(1), test_predicted.argmax(1)))

tf.Tensor(
[[   0    0    0    0    0    1    0    0    0    0    0    1    0    0
     0    0    0]
 [  65    0    0   60   62   71  290    7   57    0    0 1916    0    0
     0    0    0]
 [  48    0    0   40   51   61  216    7   54    0    0 1492    0    0
     0    0    0]
 [  10    0    0   12   14   17   49    3   11    0    0  351    0    0
     0    0    0]
 [   2    0    0    5   20    9   19    2    6    0    0  128    0    0
     0    0    0]
 [   6    0    0    1    3    7   18    0    2    0    0   72    0    0
     0    0    0]
 [   6    0    0    6   13   13   88    0    9    0    0  276    0    0
     0    0    0]
 [   2    0    0    0    0    0    0    0    0    0    0    6    0    0
     0    0    0]
 [  20    0    0   11   28   12  128    2   38    0    0  457    0    0
     0    0    0]
 [   3    0    0    2    8    1   15    0    3    0    0   71    0    0
     0    0    0]
 [   0    0    0    1    1    0    3    0    0    0    0   16    0    0
     0    0    0]

In [41]:
!zip -r /content/trainedModel_balanced.zip /content/trainedModel_balanced

  adding: content/trainedModel_balanced/ (stored 0%)
  adding: content/trainedModel_balanced/saved_model.pb (deflated 70%)
  adding: content/trainedModel_balanced/variables/ (stored 0%)
  adding: content/trainedModel_balanced/variables/variables.data-00000-of-00001 (deflated 49%)
  adding: content/trainedModel_balanced/variables/variables.index (deflated 80%)
  adding: content/trainedModel_balanced/keras_metadata.pb (deflated 91%)
  adding: content/trainedModel_balanced/fingerprint.pb (stored 0%)
  adding: content/trainedModel_balanced/assets/ (stored 0%)


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [45]:
!mv "/content/trainedModel_balanced.zip" "/content/drive/My Drive/trainedModel_balanced.zip"